# Vector Databases

#### Vector Databases stores 'semantic information' about assets, making it searchable by semantic features.

In [12]:
#r "nuget: Microsoft.SemanticKernel, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.46.0-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.45.0"

Installed Packages Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.Connectors.InMemory, 1.46.0-preview Microsoft.SemanticKernel.Plugins.OpenApi, 1.45.0

##### First step, generate embeddings.

In [ ]:
using System.ComponentModel;
using System.Threading;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Embeddings;
using Kernel = Microsoft.SemanticKernel.Kernel;
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0001
IKernelBuilder builder = Kernel.CreateBuilder();
builder.AddOpenAITextEmbeddingGeneration(
    modelId: "text-embedding-3-small",         
    apiKey: apiKey,                  
    orgId: orgId       
);
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
Kernel kernel = builder.Build();
ITextEmbeddingGenerationService  textEmbeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

string transcriptText1 = "Customer: I want to return this shoes. It is not color I ordered.";
var embeddings1 = await textEmbeddingGenerationService.GenerateEmbeddingAsync(transcriptText1, kernel, CancellationToken.None);
Transcript transcript1 = new Transcript
{
    TranscriptId = 1,
    TranscriptText = transcriptText1,
    TranscriptTextEmbedding = embeddings1
};



string transcriptText2 = "Customer: I would like to speak to a supervisor please. I am not satisfied with the service.";
var embeddings2 = await textEmbeddingGenerationService.GenerateEmbeddingAsync(transcriptText2, kernel, CancellationToken.None);
Transcript transcript2 = new Transcript
{
    TranscriptId = 2,
    TranscriptText = transcriptText2,
    TranscriptTextEmbedding = embeddings2
};


class Transcript
{
    public ulong TranscriptId { get; set; }
    public string TranscriptText { get; set; }
    public ReadOnlyMemory<float> TranscriptTextEmbedding { get; set; }
}


##### Now, we can bring in a vector store.

In [40]:
using Microsoft.SemanticKernel.Connectors.InMemory;
using Microsoft.SemanticKernel.Memory;
using Microsoft.Extensions.VectorData;

// Use InMemoryVectorStore for local testing. 
// For production, use a persistent vector store like QDrant, Vespa, or Pinecone.
var vectorStore = new InMemoryVectorStore(); 

var transcriptDefinition = new VectorStoreRecordDefinition
{
    Properties = new List<VectorStoreRecordProperty>
    {
        new VectorStoreRecordKeyProperty("TranscriptId", typeof(ulong)),
        new VectorStoreRecordDataProperty("TranscriptText", typeof(string)) { IsFullTextSearchable = true },
        new VectorStoreRecordVectorProperty("TranscriptTextEmbedding", typeof(ReadOnlyMemory<float>)) { Dimensions = 4, DistanceFunction = DistanceFunction.CosineSimilarity, IndexKind = IndexKind.Hnsw },
    }
};
IVectorStoreRecordCollection<ulong, Transcript> collection = vectorStore.GetCollection<ulong, Transcript>("Transcripts", transcriptDefinition);

await collection.CreateCollectionIfNotExistsAsync(
    CancellationToken.None
);


collection.UpsertAsync(transcript1,
    CancellationToken.None
);

collection.UpsertAsync(transcript2,
    CancellationToken.None
);




##### Now, let's search

In [41]:
var searchKey = "I am looking for the transcript where customer wants to talk to a manager";

ReadOnlyMemory<float> searchVector = await textEmbeddingGenerationService.GenerateEmbeddingAsync(
    searchKey, kernel, CancellationToken.None);

var searchResult = await collection.VectorizedSearchAsync(
    searchVector,
    new() { Top = 1 }, // number of results to return
    CancellationToken.None
);

await foreach (var result in searchResult.Results)
{
    Console.WriteLine($"TranscriptId: {result.Record.TranscriptId}");
    Console.WriteLine($"TranscriptText: {result.Record.TranscriptText}");
  
}

TranscriptId: 2
TranscriptText: Customer: I would like to speak to a supervisor please. I am not satisfied with the service.


##### Next, RAG!

##### Make your vector store available to your model through plugins or function-calling

In [44]:
// Add code to create instances of IVectorStoreRecordCollection and ITextEmbeddingGenerationService
using Microsoft.SemanticKernel.Data;
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0001

// Create a text search instance using the vector store record collection.
//var textSearch = new VectorStoreTextSearch<Transcript>(collection, textEmbeddingGenerationService);

// Build a text search plugin with vector store search and add to the kernel
//var searchPlugin = textSearch.CreateWithGetTextSearchResults("SearchPlugin");
//kernel.Plugins.Add(searchPlugin);

// Invoke prompt and use text search plugin to provide grounding information
OpenAIPromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
KernelArguments arguments = new(settings);
Console.WriteLine(await kernel.InvokePromptAsync("Why does customer want to return items?", arguments));

I'm currently unable to retrieve specific search results for why customers return items. However, I can provide general insights: 

1. **Defective Product**: Items that are damaged or not functioning as advertised can prompt returns.
2. **Incorrect Size/Color**: Discrepancies in size, color, or product specifications often lead to returns, especially in fashion.
3. **Changed Mind**: Customers may simply decide they no longer want the product after purchasing.
4. **Better Price Found Elsewhere**: Finding the same product for a lower price can result in a return.
5. **Wrong Item Delivered**: Mistakes in the shipping process can cause customers to return items.
6. **Poor Quality**: Items not meeting quality expectations may be returned by the customer.
7. **Expectation Mismatch**: If the product does not meet the customer's expectations based on the description or images, they might return it.

If you need more detailed or specific reasons, let me know, and I can try searching again or pr